In [ ]:
import glob
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from analyse_utils import parse_path
from magsac_utils import read_magsac, show_dimension, load_magsac_data
from magsac_data import Magsac_Data

In [ ]:
from analyse_utils import load_paths, create_data_container, load_data, summarize_data, generate_image, create_value_container

In [ ]:
magsac_default_path = "/home/clementriu/Documents/these/RANSAC-benchmark/merged_script/experience_data/" + \
"remote_homography/magsac_test/experience_results/uniform/hom_{}_std*_ratio*_magsac{}.txt"

In [ ]:
other_editable_glob_to_results = "/home/clementriu/Documents/these/RANSAC-benchmark/merged_script/experience_data/" + \
"remote_homography/magsac_test/experience_results/{}/hom_{}_std*_ratio*_output.txt"


In [ ]:
folder_names = ['uniform']
data_numbers = [i for i in range(1, 2)]
noise_stds = [float(i / 10) for i in range(31)]
outlier_ratios = [float(i / 10) for i in range(10)]

file_names = ["Weights", "Labels", "Errors", "ErrorsAll", "Inliers", "PosInl"]
file_types = ["float", "int", "float", "float", "int", "int"]
file_index = {file_name: index for index, file_name in enumerate(file_names)}

In [ ]:
magsac_data_for_process = load_magsac_data(data_numbers, noise_stds, outlier_ratios, file_names, file_types, file_index, magsac_default_path)

In [ ]:
other_paths = load_paths(folder_names, data_numbers, noise_stds, other_editable_glob_to_results)

In [ ]:
other_p_datas = create_data_container(outlier_ratios, noise_stds, data_numbers, folder_names)
other_r_datas = create_data_container(outlier_ratios, noise_stds, data_numbers, folder_names)
other_s_datas = create_data_container(outlier_ratios, noise_stds, data_numbers, folder_names)
other_t_datas = create_data_container(outlier_ratios, noise_stds, data_numbers, folder_names)

In [ ]:
load_data(other_p_datas,
          other_r_datas,
          other_s_datas,
          other_t_datas,
          folder_names,
          data_numbers,
          noise_stds,
          other_paths)

In [ ]:
algorithms = ["Ransac", "ORSA", "LRT"]

In [ ]:
other_precision_mean, other_precision_std = summarize_data(folder_names, data_numbers, algorithms, noise_stds, outlier_ratios, other_p_datas)
other_recall_mean, other_recall_std = summarize_data(folder_names, data_numbers, algorithms, noise_stds, outlier_ratios, other_r_datas)
other_sigma_mean, other_sigma_std = summarize_data(folder_names, data_numbers, algorithms[1:], noise_stds, outlier_ratios, other_s_datas)
other_time_mean, other_time_std = summarize_data(folder_names, data_numbers, algorithms, noise_stds, outlier_ratios, other_t_datas)

In [ ]:
magsac_p_datas_tt = create_data_container(outlier_ratios, noise_stds, data_numbers, folder_names)
magsac_r_datas_tt = create_data_container(outlier_ratios, noise_stds, data_numbers, folder_names)

In [ ]:
folder_name = folder_names[0]
for data_number in data_numbers:
    for noise_std in noise_stds:
        for outlier_ratio in outlier_ratios:
            values_to_transform = magsac_data_for_process[data_number][noise_std][outlier_ratio]
            magsac_computed_p = [value_to_transform.get_precision_from_threshold(noise_std) 
                                 for value_to_transform in values_to_transform]
            magsac_p_datas_tt[folder_name][data_number][noise_std][outlier_ratio] = [magsac_computed_p]
            magsac_computed_r = [value_to_transform.get_recall_from_threshold(noise_std) 
                                 for value_to_transform in values_to_transform]
            magsac_r_datas_tt[folder_name][data_number][noise_std][outlier_ratio] = [magsac_computed_r]

In [ ]:
magsac_algorithm = ['Magsac']

In [ ]:
magsac_precision_mean_tt, magsac_precision_std_tt = summarize_data(folder_names, data_numbers, magsac_algorithm, noise_stds, outlier_ratios, magsac_p_datas_tt)
magsac_recall_mean_tt, magsac_recall_std_tt = summarize_data(folder_names, data_numbers, magsac_algorithm, noise_stds, outlier_ratios, magsac_r_datas_tt)

In [ ]:
all_algorithms = algorithms + magsac_algorithm

In [ ]:
def concatenate_values(noise_stds, outlier_ratios, algorithms, data_numbers, folder_names, other_values, magsac_values):
    values_concat = create_value_container(len(noise_stds), len(outlier_ratios), algorithms, data_numbers, folder_names)

    for folder_name in folder_names:
            for data_number in data_numbers:
                for algo_index, algorithm in enumerate(algorithms):
                    for noise_std_index, noise_std in enumerate(noise_stds):
                        for outlier_ratio_index, outlier_ratio in enumerate(outlier_ratios):
                            if algorithm == 'Magsac':
                                exctracted_data = magsac_values[folder_name][data_number][algorithm]
                            else:
                                exctracted_data = other_values[folder_name][data_number][algorithm]
                            values_concat[folder_name][data_number][algorithm] = exctracted_data
    return values_concat

In [ ]:
all_precision_mean_tt = concatenate_values(noise_stds, 
                                           outlier_ratios,
                                           all_algorithms,
                                           data_numbers,
                                           folder_names,
                                           other_precision_mean,
                                           magsac_precision_mean_tt)
all_precision_std_tt = concatenate_values(noise_stds, 
                                           outlier_ratios,
                                           all_algorithms,
                                           data_numbers,
                                           folder_names,
                                           other_precision_std,
                                           magsac_precision_std_tt)
all_recall_mean_tt = concatenate_values(noise_stds, 
                                           outlier_ratios,
                                           all_algorithms,
                                           data_numbers,
                                           folder_names,
                                           other_recall_mean,
                                           magsac_recall_mean_tt)
all_recall_std_tt = concatenate_values(noise_stds, 
                                           outlier_ratios,
                                           all_algorithms,
                                           data_numbers,
                                           folder_names,
                                           other_recall_std,
                                           magsac_recall_std_tt)

In [ ]:
generate_image(data_numbers, outlier_ratios, all_algorithms, folder_names, noise_stds, 
               "Precision", all_precision_mean_tt, all_precision_std_tt)#, precision_image_path)

In [ ]:
generate_image(data_numbers, outlier_ratios, all_algorithms, folder_names, noise_stds, 
               "Recall", all_recall_mean_tt, all_recall_std_tt)#, precision_image_path)

In [ ]:
assert(False)

In [ ]:
noise_threshold = 1.0
outlier_ratio = 0.5
test_data = magsac_data_for_process[1][noise_threshold][outlier_ratio][5]

In [ ]:
precision_true_threshold = test_data.get_precision_from_threshold(noise_threshold)
recall_true_threshold = test_data.get_recall_from_threshold(noise_threshold)